# KerasTuner demo

`KerasTuner` is a general purpose hyperparameter tuning library. It is integrated with Keras worflows but it is not limited to them. It can be used to tune `scikit-learn` models.

## Define the search space

In the following code example, we deinfe a Keras model with two `Dense` layers. We want to tune the number of units in the first `Dense` layer. We just define an integer hyperparameter with `hp.Int('units', min_value=32, max_value=512, step=32)` whose range is from `32` to `512`inclusive. When sampling from it, the minimum step for walking through the interval is `32`.

In [3]:
import keras
from keras import layers

def build_model_with_two_dense_layers(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # define the hyperparameter
            units=hp.Int("unit", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model
    

Let us test if the model was built successfully

In [7]:
import keras_tuner

build_model_with_two_dense_layers(keras_tuner.HyperParameters())

<Sequential name=sequential_2, built=False>

There are many other types of hyperparameters as well. We can define multiple hyperparameters in the function. In the following code, we tune whether to use a `Dropout` layer with `hp.Boolean()`, tine which activation function to use with `hp.Choice`, tune the learning rate of the optimizer with `hp.Float()`.

In [9]:
def build_model_with_two_dense_layers_and_dropout(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Tune number of units
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            # Tune the activation function to use
            activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )
    # Tune whether to use dropout.
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

build_model_with_two_dense_layers_and_dropout(keras_tuner.HyperParameters())

<Sequential name=sequential_3, built=False>

As shown below, the hyperparameters are actual values. In fact, they are just functions returning actual values. For example, `hp.Int()` returns an `int` value. Therefore, you can put them into variables, `for` loops, or `if` conditions. 

In [10]:
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value=32, max_value=512, step=32))

32


The hyperparameters can be defined in advance and keep the Keras model code in a separate function as shown below

In [12]:
def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_4, built=False>

Each of the hyperparameters is uniquely identified by its name - the first argument of the corresponding hp type.
To tune the number of units in different `Dense` layers separately as different hyperparameters, we give them different names as `f"units_{i}"`.

Notably, this is also an example of creating conditional hyperparameters. There are many hyperparameters specifying the number of units in the `Dense` layers. The number of such hyperparameters is decided by the number of layers, which is also a hyperparameter.  Therefore, the total number of hyperparameters used may be different from trial to trial. Some hyperparameter is only used when a certain condition is satisfied. For example, `units_3` is only used when `num_layers` is larger than 3. With `KerasTuner`, one can easily define such hyperparameters dynamically while creating the model.

In [13]:
def build_model_conditional_hpars(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # tune the number of layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # tune number of units separately
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model
            
build_model_conditional_hpars(keras_tuner.HyperParameters())

<Sequential name=sequential_5, built=False>

## Start the search

After defining the search space, we need to select a tuner class to run the search. The available algorithms are `RandomSearch`, `BayesianOptimization`, and `Hyperband`. Here we use `RandomSearch` as an example.

To initialize the tuner we need to specify several arguments in the initializer.

* `hypermodel`: the model building function which is `build_model` in our case
* `objective`: the name of the objective to optimize. Note that whether to minimize or maximize is automatically inferred for built-in metrics.
* `max_trials`: the total number of trials to run during the search
* `executions_per_trial`: the number of models that should be built and fit for each trial. Different trialshave different hyperparameter values. The executions within the same trial have the same hyperparameter values. The purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If we want to get results faster we could set `executions_per_trial=1` (single round of training for each model configuration)
* `overwrite`: control whether to overwrite the previous results in the same directory or resume the previous search instead. Here we set `overwrite=True` to start a new search and ignore any previous results.
* `directory`: a path to a directory for storing the search results.
* `project_name`: the name of the sub-directory in the `directory`.

In [15]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model_conditional_hpars,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

Printing summary of the search space:

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


## Preparation of the MNIST dataset for the search

In [17]:
import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
